# Homework 18: Maximum Likelihood

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import utils as utils
from fractions import Fraction

from IPython.display import display, HTML

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

utils.load_custom_styles()

---
## Exercise 3.1

<img src="figures/homework-18/exercise-3.1.png" width="600" />




















In [2]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])

Compute the prior probabilities for the classes. We can compute the prior probability for each class $c_k$ as follows:

$$
P(c_k) = \frac{N_k}{\sum_l^K N_l}
$$
where $N_k$ is the number of samples in class $c_k$

Class $c_1$ has 7 samples and $c_2$ has 5 samples. In total, we have 12 samples. Therefore, $P(c_1) = 7/12=0.58$ and $P(c_2) = 5/12=0.42$.

In [3]:
N1 = c1.shape[1]
N2 = c2.shape[1]
N_sum = N1 + N2

P_c1 = N1/N_sum
P_c2 = N2/N_sum
P_c1, P_c2

We compute the class-conditional probabilities in multiple steps:
$$
p(\mathbf{x} \mid c_k) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)  }
  {  \sum_{l=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_l \rVert_2)   }
$$

Step 1: Compute the mean vector associated with each class:

In [4]:
m1 = np.mean(c1, axis=1).reshape(-1, 1)
m1

array([[-0.85714286],
       [-0.9       ]])

In [5]:
m2 = np.mean(c2, axis=1).reshape(-1, 1)
m2

array([[1.1],
       [1. ]])

Step 2: Compute $\exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)$

In [6]:
c1_denominator = np.exp(-np.linalg.norm(X - m1, axis=0))
c1_denominator

array([0.28855859, 0.07016722, 0.40201441, 0.18136432, 0.0830477 ])

Step 3: compute $\exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)$:

In [7]:
c2_denominator = np.exp(-np.linalg.norm(X - m2, axis=0))
c2_denominator

array([0.22613867, 0.90483742, 0.09769151, 0.2822644 , 0.24836923])

Step 4: Compute the class-condition probability for class 1:

$$
p(\mathbf{x} \mid c_1) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)  }
  {  \sum_{k=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)   }
$$

In [8]:
p_x_c1 = c1_denominator / (c1_denominator + c2_denominator)
p_x_c1

array([0.5606375 , 0.07196604, 0.804502  , 0.39118439, 0.25058376])

Step 5: Compute the class-condition probability for class 2:

$$
p(\mathbf{x} \mid c_2) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_2 \rVert_2)  }
  {  \sum_{k=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)   }
$$

In [9]:
p_x_c2 = c2_denominator / (c1_denominator + c2_denominator)
p_x_c2

array([0.4393625 , 0.92803396, 0.195498  , 0.60881561, 0.74941624])

Step 6: Compute $p(\mathbf{x})$:
$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x} \mid c_k) P(c_k)
$$


In [10]:
p_x = (p_x_c1 * P_c1) + (p_x_c2 * P_c2)
p_x

array([0.51010625, 0.42866101, 0.55075033, 0.48186407, 0.45843063])

Step 7: We can now compute conditional probability, which has the following formula (Bayes' formula):

$$
P(c_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid c_k) P(c_k)}{p(\mathbf{x})}
$$
where
$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x} \mid c_k) P(c_k)
$$

In [11]:
P_c1_x = p_x_c1 * P_c1 / p_x
P_c1_x

array([0.64111848, 0.0979333 , 0.85209723, 0.47355865, 0.3188571 ])

In [12]:
P_c2_x = p_x_c2 * P_c2 / p_x
P_c2_x

array([0.35888152, 0.9020667 , 0.14790277, 0.52644135, 0.6811429 ])

<div class="warning">
    What!? my conditional probability is different than the one presented in the solution manual.
</div>

Step 8: Once we have the condition probabilities, we can use Bayes' Decision Rule to classify our samples $X$:

In [13]:
# Create a matrix so we can easily compute argmin
P_ck_x = np.concatenate([[P_c1_x], 
                         [P_c2_x]])
P_ck_x

array([[0.64111848, 0.0979333 , 0.85209723, 0.47355865, 0.3188571 ],
       [0.35888152, 0.9020667 , 0.14790277, 0.52644135, 0.6811429 ]])

In [14]:
np.argmax(P_ck_x, axis=0)+1

array([1, 2, 1, 2, 2])

So our classification is as follows: 

\begin{align}
c_1 &= \{ \mathbf{x}_1, \mathbf{x}_3 \} \\
c_2 &= \{ \mathbf{x}_2, \mathbf{x}_4, \mathbf{x}_5 \}
\end{align}

  ---
## Exercise 3.2

<img src="figures/homework-18/exercise-3.2.png" width="600" />




















In [15]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])
Lambda = np.array([[0, 1],
                   [1, 0]])

In [16]:
N1 = c1.shape[1]
N2 = c2.shape[1]
N_sum = N1 + N2

# Compute P(c_k)
P_c1 = N1/N_sum
P_c2 = N2/N_sum

# Compute mean for each class
m1 = np.mean(c1, axis=1).reshape(-1, 1)
m2 = np.mean(c2, axis=1).reshape(-1, 1)

# Compute p(x|c_k)
c1_denominator = np.exp(-np.linalg.norm(X - m1, axis=0))
c2_denominator = np.exp(-np.linalg.norm(X - m2, axis=0))
p_x_c1 = c1_denominator / (c1_denominator + c2_denominator)
p_x_c2 = c2_denominator / (c1_denominator + c2_denominator)

# Compute p(x)
p_x = (p_x_c1 * P_c1) + (p_x_c2 * P_c2)

# Compute P(c_k|x)
P_c1_x = (p_x_c1 * P_c1) / p_x
P_c2_x = (p_x_c2 * P_c2) / p_x

# Compute R(c_k|x)
R_c1_x = Lambda[0,0] * P_c1_x + Lambda[0,1] * P_c2_x
R_c2_x = Lambda[1,0] * P_c1_x + Lambda[1,1] * P_c2_x

In [17]:
R_c1_x

array([0.35888152, 0.9020667 , 0.14790277, 0.52644135, 0.6811429 ])

In [18]:
R_c2_x

array([0.64111848, 0.0979333 , 0.85209723, 0.47355865, 0.3188571 ])

In [19]:
R_ck_x = np.concatenate([[R_c1_x], 
                         [R_c2_x]])
np.argmin(R_ck_x, axis=0)+1

array([1, 2, 1, 2, 2])

So our classification is as follows: 

\begin{align}
c_1 &= \{ \mathbf{x}_1, \mathbf{x}_3 \} \\
c_2 &= \{ \mathbf{x}_2, \mathbf{x}_4, \mathbf{x}_5 \} \\
\end{align}

---
## Exercise 3.3

<img src="figures/homework-18/exercise-3.3.png" width="600" />







---
## Exercise 3.4

<img src="figures/homework-18/exercise-3.4a.png" width="600" />





<img src="figures/homework-18/exercise-3.4b.png" width="600" />














---
## Exercise 3.4

<img src="figures/homework-18/exercise-3.5.png" width="600" />











